# Example 2: Iterative optimization of a function, which can be sampled

In this notebook the framework is used to iteratively optimize a function which can be sampled. For simplicity we will use a known function to illustrate, but it is not a requirement that the function can be written down explicitly as we do below. Hence, this option of using the framework allows for optimizing _unknown_ functions or _functions without explicit definitions_ and will come in handy for e.g. optimizing complex experiments. 

In the following we will use the same function as for Example 1
$$
f(x) = - (6x - 2)^2 \sin{(12 x - 4 )}, \quad x \in [0; 1].
$$

It is known that the function above has its maximum at $x^* = 0.75725$; the only covariate is $x$.


## Framework approach

We will leverage the iterative optimization approach of the framework which requires using the methods `.ask` and `.tell`. This approach allows for iterative optimization and optimization of any callable function, known or otherwise.

The optimization process can be stopped after any number of iterations.


## Technical note

Installation of `torch` and `torchvision` (required dependencies) cannot be bundled as part of the `creative_project` installable. This is unfortunate, but a known issue it seems. Therefore these must be installed first, before installing `creative_project`.

In [ ]:
# Preamble

# Install torch and torchvision. Use this link to identify the right versions to install on your system 
# depending on configuration: https://pytorch.org/get-started/locally/

pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

# Install creative_project from github (will require authentication with password)
pip install --user https://github.com/svedel/kre8_core/

In [ ]:
# import
import numpy as np
import torch
from creative_project import CreativeProject
import matplotlib.pyplot as plt
%matplotlib inline

Define the problem to optimize. Here we define a known function, but it's not necessary that the function to optimize can be written down; all that's needed is that it can be sampled

In [ ]:
def f(x):
    return -(6*x - 2)**2*torch.sin(12*x-4)

x = torch.linspace(0,1)
plt.figure(figsize=(8, 4))
plt.plot(x.numpy(), f(x).numpy())
plt.show()

Set up the problem

In [ ]:
# define the range of the covariate 'x'
x_exp = 0.5  # start point for x (a priori expectation)
x_min = 0  # minimum allowed value of x
x_max = 1  # maximum allowed value of x
x_input=[(x_exp, x_min, x_max)]

In [ ]:
# initialize class instance
cc = CreativeProject(covars=x_input, model="Custom", nu=2.5)

Define the loop to solve iteratively. Here we use the option of adding the sampled covariates and associated response of the system (what we're using the function `f` for defined above) as input to `.tell`. If not added directly, `.tell` will prompt the user to provide sampled covariate points and responses at each iteration.

In [ ]:
# iteration limit
max_iter = 21

# run the solution
for i in range(max_iter):

    # generate candidate
    cc.ask()

    # sample response
    covars = torch.tensor([[cc.proposed_X[-1].item()]], dtype=torch.double)
    response = f(covars)

    # report response
    cc.tell(covars=covars, response=response)

Best guess after solving

In [ ]:
cc.current_best()

Plot the solution

In [ ]:
cc.plot_1d_latest()